In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3
import glob
import sys
sys.path.insert(0, "/mnt/ssd1/Documents/Vivek/MalkoFish/MalkoAnalyzer")
import swarm
import BaseToolbox as bt
import plotBox as pb
import plotter
import copy
import cv2
from imp import reload
reload(pb)
reload(plotter)
reload(bt)

plotter.pb.boxSize.defineBoxSize((-5,5),(-5,5),(-1,1)) 
plotter.pb.boxSizeRelative.defineBoxSize((-10,10),(-10,10),(-1,1))

In [2]:
dataDIR = '/mnt/ssd1/Documents/Vivek/flyVR/fly-matrix/data/biasdata/'
projectDB = '/mnt/ssd1/Documents/Vivek/flyVR/fly-matrix/dbGen/biasProjects.db'
expDB = '/mnt/ssd1/Documents/Vivek/flyVR/fly-matrix/dbGen/biasExperiments.db'

In [3]:
nPosts = 2

In [4]:
def rotate(xy, radians):
    x, y = xy
    c, s = np.cos(radians), np.sin(radians)
    j = np.matrix([[c, s], [-s, c]])
    m = np.dot(j, [x, y])

    return m

In [5]:
# establish a connecttion to the project database
conn = sqlite3.connect(projectDB)
# connect a cursor that goes through the project database
cursorProject = conn.cursor()
# establish a second connecttion to the experiment database
conn2 = sqlite3.connect(expDB)
# connect a cursor that goes through the experiment database
cursorExperiment = conn2.cursor()

# pick experiments from specified project
cursorExperiment.execute("Select expId from experiments where project = ? and exp >= ? and exp < ?",('DecisionBias', (nPosts-2)*10, (nPosts-1)*10))
fetched = cursorExperiment.fetchall()
print('fetched : ' + str(fetched))

fetched : [('1f7fead4-66e1-4103-a19c-a59b8ee2294f',), ('a61c4515-d5cf-4546-9a9a-4b990669581e',), ('f487c95f-7bc1-4902-b3e8-ad00e27904d5',), ('a8600224-8e5e-4139-a459-40df4dff9c9d',), ('0de4d57a-f638-4b7b-9165-acaf578d53b4',), ('fa51b041-dab5-462a-aef2-d36ed6d1569f',), ('936878d6-bc01-4ccf-94fc-eb70a5886c4d',), ('b23a5fad-36ac-4847-8987-774dd8873c44',), ('8d544b52-6e44-4271-9a8d-d31863c3af76',), ('9121e08e-d431-4962-ad94-e39eb7240eb8',), ('468187d1-41a1-43d2-8b00-fa239d2b2657',), ('5e2a883f-e041-4424-b226-fae093bd5adf',), ('1555d583-7743-4b1a-a19e-9610eddcf819',), ('023bf76b-842d-4bd3-b107-d943f9dadf6a',), ('0470f38a-9359-42ac-a719-d4353898269c',), ('1b9684ec-6dee-4f65-853b-03ba4665814a',), ('50f0a4b5-cc2d-4c05-97b6-0e61f48ee43f',), ('04865b04-3cfd-48f6-80af-9be13c7f2a0f',), ('f0e62385-8471-4ed4-9e47-2e28853d66e2',)]


In [6]:
angles = []
dataDict0 = []
dataDict1 = []
rot_post0 = []
for uuid in range(0,len(fetched)):
    cursorExperiment.execute("Select exp from experiments where expId = ?", (fetched[uuid][0],))
    fetch_exp = cursorExperiment.fetchall()[0][0]
    cursorExperiment.execute("Select replicate from experiments where expId = ?", (fetched[uuid][0],))
    fetch_rep = cursorExperiment.fetchall()[0][0]

    cursorProject.execute("Select post0 from projects where project = ? and exp = ? and replicate = ?",('DecisionBias', fetch_exp, fetch_rep))
    dataDict0.append(cursorProject.fetchall())
    cursorProject.execute("Select post1 from projects where project = ? and exp = ? and replicate = ?",('DecisionBias', fetch_exp, fetch_rep))
    dataDict1.append(cursorProject.fetchall())

    tmp = pd.read_csv(dataDIR + fetched[uuid][0] + "/results.csv", names = ["x", "y", "z", "dir", "event", "t", "nStimuli"])
    tmp['uuid'] = uuid
    tmp['post0_x'] = tmp['post0_y'] = tmp['post1_x'] = tmp['post1_y'] = tmp['angle'] = np.nan

    df = tmp if uuid == 0 else pd.concat([df,tmp])


In [7]:
# add post positions to dataframe
nStimuli = np.unique(df['nStimuli'])
for uuid in range(0,len(fetched)):
    for n in nStimuli:
        if type(eval(dataDict0[uuid][n][0])) != type(None):
            df.loc[(df['uuid'] == uuid) & (df['nStimuli'] == n),'post0_x'] = eval(dataDict0[uuid][n][0])['position'][0]
            df.loc[(df['uuid'] == uuid) & (df['nStimuli'] == n),'post0_y'] = eval(dataDict0[uuid][n][0])['position'][1]
            df.loc[(df['uuid'] == uuid) & (df['nStimuli'] == n),'angle'] = eval(dataDict0[uuid][n][0])['angle']
        
        if type(eval(dataDict1[uuid][n][0])) != type(None):
            df.loc[(df['uuid'] == uuid) & (df['nStimuli'] == n),'post1_x'] = eval(dataDict1[uuid][n][0])['position'][0]
            df.loc[(df['uuid'] == uuid) & (df['nStimuli'] == n),'post1_y'] = eval(dataDict1[uuid][n][0])['position'][1]
            df.loc[(df['uuid'] == uuid) & (df['nStimuli'] == n),'angle'] = eval(dataDict1[uuid][n][0])['angle']

In [8]:
p0_dist = np.sqrt((df['x'] - df['post0_x'])**2 + (df['y'] - df['post0_y'])**2)
p1_dist = np.sqrt((df['x'] - df['post1_x'])**2 + (df['y'] - df['post1_y'])**2)

df['dmin'] = np.nanmin([p0_dist, p1_dist], axis=0)
df['post'] = 0
df.loc[p1_dist < p0_dist, 'post'] = 1

In [9]:
tmax = df.loc[:,['uuid', 'nStimuli', 'event', 't']]
tmax = tmax.groupby(['uuid', 'nStimuli', 'event']).max().reset_index()
dmin = df.loc[:,['uuid', 'nStimuli', 'event', 't', 'dmin', 'post', 'angle']]

dists = pd.merge(tmax, dmin, how='left')
dists = dists[dists['dmin'] < 0.8]

dists = dists[(dists['nStimuli'] > 0) & (dists['nStimuli'] < 4)]

In [10]:
len(dists[(dists['angle'] == np.unique(dists['angle'])[0]) & (dists['post'] == 1)]) / len(dists[dists['angle'] == np.unique(dists['angle'])[0]])

0.5666666666666667

In [11]:
len(dists[(dists['angle'] == np.unique(dists['angle'])[1]) & (dists['post'] == 1)]) / len(dists[dists['angle'] == np.unique(dists['angle'])[1]])

0.56

In [12]:
len(dists[(dists['angle'] == np.unique(dists['angle'])[2]) & (dists['post'] == 1)]) / len(dists[dists['angle'] == np.unique(dists['angle'])[2]])

0.6515151515151515